In [4]:
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import StandardScaler
import talib

In [230]:
def stochastic_K(close, high, low, n):
    return [np.nan]*n+[(close[i] - min(low[i-n+1:i+1]))/(max(high[i-n+1:i+1]-min(low[i-n+1:i+1])))*100 for i in range(10, len(close))]

def AD_Osc(high, low, close):
    return (high[1:]-close[:-1]) / ((high[1:]-low[1:])+0.000001)

def tech_indi_1(df):
    n = 10
    open = df['open']
    high = df['high']
    close = df['close']
    low = df['low']

    df['SMA'] = talib.SMA(close, n)
    df['WMA'] = talib.WMA(close, n)
    df['MOM'] = talib.MOM(close, n)
    df['K'] = stochastic_K(close, high, low, n)
    df['D'] = talib.SMA(df['K'], n)
    df['RSI'] = talib.RSI(close, n)
    df['MACD'], macdsignal, macdhist = talib.MACD(close, 12, 26, n)
    df['WILLR'] = talib.WILLR(high, low, close, n)
    df['AD'] = AD_Osc(high, low, close)
    df['CCI'] = talib.CCI(high, low, close, n)

    return df.dropna()

def tech_indi_2(df):
    df_2 = df.copy()

    df_2['SMA'] = list(map(lambda x: '+1' if x[0] > x[1] else '-1', zip(df['close'],df['SMA'])))
    df_2['WMA'] = list(map(lambda x: '+1' if x[0] > x[1] else '-1', zip(df['close'],df['WMA'])))
    df_2['MOM'] = list(map(lambda x: '+1' if x >= 0 else '-1', df['MOM']))
    df_2['RSI'] =  [np.nan] + list(map(lambda x: '+1' if x[0] <= 30 else ('-1' if x[0] >= 70 else ('+1' if x[0] > x[1] else '-1')), zip(df['RSI'][1:], df['RSI'][:-1])))
    df_2['CCI'] =  [np.nan] + list(map(lambda x: '+1' if x[0] >= 200 else ('-1' if x[0] <= -200 else ('+1' if x[0] > x[1] else '-1')), zip(df['CCI'][1:], df['CCI'][:-1])))

    def trans(col):
        return [np.nan] + list(map(lambda x: '+1' if x[0] >= x[1] else '-1', zip(df[col][1:], df[col][:-1])))
    df_2['K'] = trans('K')
    df_2['D'] = trans('D')
    df_2['MACD'] = trans('MACD')
    df_2['WILLR'] = trans('WILLR')
    df_2['AD'] = trans('AD')

    return df_2.iloc[:, 5:].dropna()

In [237]:
df_raw = pd.read_csv('data.csv', index_col = 0, header=1)
df_raw.dropna(inplace=True)
df.index = pd.to_datetime(df.index)
col_name = ['open', 'high', 'low', 'close', 'vol']
df_TX = df_raw.iloc[:,:5].set_axis(col_name, axis = 1)
df_0050 = df_raw.iloc[:,5:10].set_axis(col_name, axis = 1)
df_2330 = df_raw.iloc[:,10:15].set_axis(col_name, axis = 1)
df_2881 = df_raw.iloc[:,15:20].set_axis(col_name, axis = 1)

In [ ]:
indi = ['SMA', 'WMA', 'MOM', 'K', 'D', 'RSI', 'MACD', 'WILLR', 'AD', 'CCI']
dfs = [df_TX, df_0050, df_2330, df_2881]

[df_TX, df_0050, df_2330, df_2881] = list(map(tech_indi_1, dfs))

dfs = [df_TX, df_0050, df_2330, df_2881]
data_2= [df_TX_2, df_0050_2, df_2330_2, df_2881_2] = list(map(tech_indi_2, dfs))

dfs = [df_TX_1, df_0050_1, df_2330_1, df_2881_1] = list(map(pd.DataFrame, list(map(StandardScaler().fit_transform, dfs))))
dfs = [df_TX_1, df_0050_1, df_2330_1, df_2881_1] = list(map(lambda x: x.iloc[:, 5:], dfs))
data_1 = []
for d in dfs:
    d = d.set_axis(df_TX.index, axis = 0)
    data_1.append(d.set_axis(indi, axis = 1))

data_1 -> method 1  
data_2 -> method 2

index: { 0 :  'TX',  1 : '0050',  2 : '2330',  3 : '2881'}    

In [243]:
index = { 0 :  'TX',  1 : '0050',  2 : '2330',  3 : '2881'}    
for i, d in enumerate(data_1):
    d.to_csv('method_1_'+index[i]+'.csv', index_label='Date', columns=indi)

for i, d in enumerate(data_2):
    d.to_csv('method_2_'+index[i]+'.csv', index_label='Date', columns=indi)